# Generate a single [character.ai](https://beta.character.ai/) character definition

This example shows how to generate the character definition of a single [character.ai](https://beta.character.ai/) character from a corpus. For the corpus in this example, we use the movie transcript of [Thor: Love and Thunder (2022)](https://scrapsfromtheloft.com/movies/thor-love-and-thunder-transcript/).

To generate your own character definition:
1. Put the corpus into a single a `.txt` file inside the `data/` directory.
2. Assign the name of the `.txt` file to the `CORPUS` constant below.
3. Assign the name of the character you want to generate description for to `CHARACTER_NAME` constant below.
4. Run this notebook.

In [7]:
CORPUS = 'data/thor_love_and_thunder.txt'
CHARACTER_NAME = "Jane Foster"  # the name of the character we want to generate a description for

In [8]:
from dataclasses import asdict
import json
import os

from data_driven_characters.character import get_character_definition
from data_driven_characters.corpus import get_characters, get_corpus_summaries, load_docs

In [9]:
# create directories to cache results and intermediate outputs
SUMMARY_TYPE = "map_reduce"  # summarize each chunk of the corpus independently
OUTPUT_ROOT = "output"
corpus_name = os.path.splitext(os.path.basename(CORPUS))[0]
output_dir = f"{OUTPUT_ROOT}/{corpus_name}/summarytype_{SUMMARY_TYPE}"
os.makedirs(output_dir, exist_ok=True)
summaries_dir = f"{output_dir}/summaries"
character_definitions_dir = f"{output_dir}/character_definitions"
os.makedirs(character_definitions_dir, exist_ok=True)

## Summarization
Because the entire corpus does not fit in the context length of the LLM, we split it into a list of chunks.
We turn the list of chunks into a list of summaries using one of [LangChain's summarization chains](https://langchain-langchain.vercel.app/docs/modules/chains/document/).

If `SUMMARY_TYPE = 'refine'`, we first summarize the first chunk, and then each subsequent summary is generated from the previous summary and the current chunk.
If `SUMMARY_TYPE = 'map_reduce'`, we summarize each chunk independently.

Because the summaries are expensive to generate, they are cached in `summaries_dir`.

In [10]:
# split corpus into a set of chunks
docs = load_docs(
    corpus_path=CORPUS,
    chunk_size=2048,  # number of tokens per chunk
    chunk_overlap=64,  # number of tokens of overlap between chunks
)

# generate summaries
corpus_summaries = get_corpus_summaries(docs=docs, summary_type=SUMMARY_TYPE, cache_dir=summaries_dir)

Summaries already exist. Loading summaries.


## Generate [character.ai](https://beta.character.ai/) character definition
Based on the corpus, we can now generate the elements - name, short description (50 characters), long description (500 characters), and custom greeting - that are required to [create a character.ai character](https://beta.character.ai/editing). These character definitions are cached in `character_definitions_dir`.

In [11]:
character_definition = get_character_definition(
        name=CHARACTER_NAME,
        corpus_summaries=corpus_summaries,
        cache_dir=character_definitions_dir,
    )

In [12]:
print(json.dumps(asdict(character_definition), indent=4))

{
    "name": "Jane Foster",
    "short_description": "I'm Jane Foster, fighting cancer & evil.",
    "long_description": "I am Jane Foster, a determined woman battling stage four cancer, yet fighting alongside Thor against the evil God Butcher, Gorr. My deep connection with Thor strengthens our resolve. As the Mighty Thor, I wield Mjolnir, despite its draining effect. Fiercely independent, I refuse help from close friends. My unshakable belief in our mission drives me to make sacrifices for others. Together, Thor and our team confront our pasts and fight to restore peace in the cosmos.",
    "greeting": "Hi there, I'm Jane. Ready to take on whatever challenges come our way?"
}
